In [ ]:
!pip install mediapipe opencv-python

In [ ]:
!pip install tensorflow


In [ ]:
!pip install numpy pandas matplotlib seaborn scikit-learn

In [ ]:
!pip install seaborn


In [ ]:
!pip install scikit-learn

In [2]:
import cv2
import mediapipe as mp
import tensorflow as tf

In [3]:
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [4]:
LINE_COLOR = (0, 255, 0)  # Green for lines
DOT_COLOR = (0, 0, 255)   # Red for dots

In [5]:
FACE_POINTS = [
    33, 133, 362, 263, 1, 9, 61, 291, 13, 14, 234, 454, 168, 93, 323, 
    199, 175, 152, 21, 22, 206, 426, 405, 18, 287, 10, 57, 287, 314,
    17, 18, 122, 351, 11  # Additional points including cheeks and jawline
]

In [6]:
def count_raised_fingers(hand_landmarks):
    fingers = []
    
    # Thumb: Check if it's raised.
    if hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x < hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_IP].x:
        fingers.append(1)  # Thumb raised
    else:
        fingers.append(0)  # Thumb down
    
    # Other fingers: Check if each fingertip is higher than its PIP.
    for i in range(1, 5):  # Index to Pinky
        if hand_landmarks.landmark[mp_hands.HandLandmark(i * 4)].y < hand_landmarks.landmark[mp_hands.HandLandmark(i * 4 - 2)].y:
            fingers.append(1)  # Finger raised
        else:
            fingers.append(0)  # Finger down
            
    return fingers.count(1)  # Count raised fingers

In [7]:
def get_hand_gesture_text(raised_fingers):
    gestures = {
        1: "approval",
        2: "victory",
        3: "shocker",
        4: "number 4",
        5: "hello"
        
    }
    return gestures.get(raised_fingers, "")

In [8]:
def is_mouth_open(face_landmarks):
    mouth_top = face_landmarks.landmark[13].y  # Upper lip point
    mouth_bottom = face_landmarks.landmark[14].y  # Lower lip point
    return (mouth_bottom - mouth_top) > 0.04  # Threshold for mouth opening


In [9]:
def draw_hand_landmarks_custom(image, hand_landmarks):
    mp_drawing.draw_landmarks(
        image,
        hand_landmarks,
        mp_hands.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=LINE_COLOR, thickness=1, circle_radius=2),
        mp_drawing.DrawingSpec(color=DOT_COLOR, thickness=3)
    )

In [10]:
def draw_face_mesh_custom(image, face_landmarks):
    for i in range(len(FACE_POINTS) - 1):
        start = face_landmarks.landmark[FACE_POINTS[i]]
        end = face_landmarks.landmark[FACE_POINTS[i + 1]]
        start_point = (int(start.x * image.shape[1]), int(start.y * image.shape[0]))
        end_point = (int(end.x * image.shape[1]), int(end.y * image.shape[0]))
        cv2.line(image, start_point, end_point, LINE_COLOR, 1)
    
    # Draw each of the selected face points as small red dots
    for idx in FACE_POINTS:
        point = face_landmarks.landmark[idx]
        point_coords = (int(point.x * image.shape[1]), int(point.y * image.shape[0]))
        cv2.circle(image, point_coords, 2, DOT_COLOR, -1)

In [11]:
def display_text_with_box(image, text, position, box_color=(50, 50, 50), text_color=(255, 255, 255)):
    (text_width, text_height), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
    box_coords = ((position[0] - 10, position[1] - 40), (position[0] + text_width + 10, position[1] + text_height - 10))
    cv2.rectangle(image, box_coords[0], box_coords[1], box_color, cv2.FILLED)
    cv2.putText(image, text, position, cv2.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)

In [ ]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands, \
     mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, min_detection_confidence=0.5) as face_mesh, \
     mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip the image horizontally for a mirror effect.
        frame = cv2.flip(frame, 1)
        
        # Convert the color space from BGR to RGB.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame for hands, face mesh, and pose for shoulders.
        hand_results = hands.process(rgb_frame)
        face_results = face_mesh.process(rgb_frame)
        pose_results = pose.process(rgb_frame)
        
        total_raised_fingers = 0
        mouth_open = False
        
        # Draw face landmarks if detected.
        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                draw_face_mesh_custom(frame, face_landmarks)
                mouth_open = is_mouth_open(face_landmarks)
        
        # Draw hand landmarks and count raised fingers if hands are present.
        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                draw_hand_landmarks_custom(frame, hand_landmarks)
                
                # Count raised fingers for each hand.
                raised_fingers = count_raised_fingers(hand_landmarks)
                total_raised_fingers += raised_fingers
        
        # Display gesture based on total raised fingers.
        sign_text = get_hand_gesture_text(total_raised_fingers)
        
        if sign_text:
            display_text_with_box(frame, sign_text, (20, 65), box_color=(50, 50, 50), text_color=(255, 255, 255))
        
        # Display "How it is going" if mouth is open.
        if mouth_open:
            display_text_with_box(frame, "How it is going", (300, 65), box_color=(0, 128, 0), text_color=(0, 255, 0))

        # Draw shoulder points if pose detected.
        if pose_results.pose_landmarks:
            left_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = pose_results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
            left_shoulder_coords = (int(left_shoulder.x * frame.shape[1]), int(left_shoulder.y * frame.shape[0]))
            right_shoulder_coords = (int(right_shoulder.x * frame.shape[1]), int(right_shoulder.y * frame.shape[0]))
            
            # Draw shoulder points as red dots.
            cv2.circle(frame, left_shoulder_coords, 4, DOT_COLOR, -1)
            cv2.circle(frame, right_shoulder_coords, 4, DOT_COLOR, -1)
            cv2.line(frame, left_shoulder_coords, right_shoulder_coords, LINE_COLOR, 1)
        
        # Display the final frame.
        cv2.imshow("Enhanced Hand, Face, and Shoulder Tracking", frame)
        
        # Exit on pressing 'q'.
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()